Option chains
=======

In [1]:
from ib_insync import *
util.startLoop()

ib = IB()
ib.connect('127.0.0.1', 7497, clientId=12)

<IB connected to 127.0.0.1:7497 clientId=12>

Suppose we want to find the options on the SPX, with the following conditions:

* Use the next three monthly expiries;
* Use strike prices within +- 20 dollar of the current SPX value;
* Use strike prices that are a multitude of 5 dollar.

To get the current market value, first create a contract for the underlyer (the S&P 500 index):

In [2]:
spx = Index('SPX', 'CBOE')
ib.qualifyContracts(spx)

[Index(conId=416904, symbol='SPX', exchange='CBOE', currency='USD', localSymbol='SPX')]

To avoid issues with market data permissions, we'll use delayed data:

In [3]:
ib.reqMarketDataType(4)

Then get the ticker. Requesting a ticker can take up to 11 seconds.

In [4]:
[ticker] = ib.reqTickers(spx)
ticker

Ticker(contract=Index(conId=416904, symbol='SPX', exchange='CBOE', currency='USD', localSymbol='SPX'), time=datetime.datetime(2024, 6, 3, 15, 31, 36, 561042, tzinfo=datetime.timezone.utc), marketDataType=3, minTick=0.01, bid=-1.0, bidSize=0.0, ask=-1.0, askSize=0.0, last=5271.67, lastSize=1.0, high=5302.11, low=5262.25, close=5277.51)

Take the current market value of the ticker:

In [5]:
spxValue = ticker.marketPrice()
spxValue

5271.67

The following request fetches a list of option chains:

In [6]:
chains = ib.reqSecDefOptParams(spx.symbol, '', spx.secType, spx.conId)

util.df(chains)

,exchange,underlyingConId,tradingClass,multiplier,expirations,strikes
0,SMART,416904,SPX,100,"[20240620, 20240718, 20240815, 20240919, 20241...","[200.0, 400.0, 500.0, 600.0, 800.0, 1000.0, 12..."
1,IBUSOPT,416904,SPX,100,"[20240620, 20240718, 20240815, 20240919, 20241...","[200.0, 400.0, 500.0, 600.0, 800.0, 1000.0, 12..."
2,CBOE,416904,SPXW,100,"[20240603, 20240604, 20240605, 20240606, 20240...","[200.0, 400.0, 600.0, 800.0, 1000.0, 1200.0, 1..."
3,IBUSOPT,416904,SPXW,100,"[20240603, 20240604, 20240605, 20240606, 20240...","[200.0, 400.0, 600.0, 800.0, 1000.0, 1200.0, 1..."
4,SMART,416904,SPXW,100,"[20240603, 20240604, 20240605, 20240606, 20240...","[200.0, 400.0, 600.0, 800.0, 1000.0, 1200.0, 1..."
5,CBOE,416904,SPX,100,"[20240620, 20240718, 20240815, 20240919, 20241...","[200.0, 400.0, 500.0, 600.0, 800.0, 1000.0, 12..."


These are four option chains that differ in ``exchange`` and ``tradingClass``. The latter is 'SPX' for the monthly and  'SPXW' for the weekly options. Note that the weekly expiries are disjoint from the monthly ones, so when interested in the weekly options the monthly options can be added as well.

In this case we're only interested in the monthly options trading on SMART:

In [7]:
chain = next(c for c in chains if c.tradingClass == 'SPX' and c.exchange == 'SMART')
chain

OptionChain(exchange='SMART', underlyingConId='416904', tradingClass='SPX', multiplier='100', expirations=['20240620', '20240718', '20240815', '20240919', '20241017', '20241114', '20241219', '20250116', '20250220', '20250320', '20250416', '20250515', '20250618', '20251218', '20261217', '20271216', '20281214', '20291220'], strikes=[200.0, 400.0, 500.0, 600.0, 800.0, 1000.0, 1200.0, 1400.0, 1600.0, 1700.0, 1800.0, 1900.0, 2000.0, 2100.0, 2200.0, 2300.0, 2350.0, 2400.0, 2450.0, 2500.0, 2550.0, 2600.0, 2650.0, 2675.0, 2700.0, 2725.0, 2750.0, 2775.0, 2800.0, 2825.0, 2850.0, 2875.0, 2900.0, 2925.0, 2950.0, 2975.0, 3000.0, 3025.0, 3050.0, 3075.0, 3100.0, 3125.0, 3150.0, 3175.0, 3200.0, 3225.0, 3250.0, 3275.0, 3300.0, 3325.0, 3350.0, 3375.0, 3400.0, 3425.0, 3450.0, 3475.0, 3500.0, 3525.0, 3550.0, 3575.0, 3600.0, 3625.0, 3650.0, 3675.0, 3700.0, 3725.0, 3750.0, 3775.0, 3800.0, 3825.0, 3850.0, 3875.0, 3900.0, 3925.0, 3950.0, 3975.0, 4000.0, 4010.0, 4020.0, 4025.0, 4030.0, 4040.0, 4050.0, 4060.0, 

What we have here is the full matrix of expirations x strikes. From this we can build all the option contracts that meet our conditions:

In [8]:
strikes = [strike for strike in chain.strikes
        if strike % 5 == 0
        and spxValue - 20 < strike < spxValue + 20]
expirations = sorted(exp for exp in chain.expirations)[:3]
rights = ['P', 'C']

contracts = [Option('SPX', expiration, strike, right, 'SMART', tradingClass='SPX')
        for right in rights
        for expiration in expirations
        for strike in strikes]

contracts = ib.qualifyContracts(*contracts)
len(contracts)

48

In [9]:
contracts[0]

Option(conId=689492328, symbol='SPX', lastTradeDateOrContractMonth='20240620', strike=5255.0, right='P', multiplier='100', exchange='SMART', currency='USD', localSymbol='SPX   240621P05255000', tradingClass='SPX')

Now to get the market data for all options in one go:

In [ ]:
tickers = ib.reqTickers(*contracts)

tickers[0]

Error 10090, reqId 61: Part of requested market data is not subscribed. Subscription-independent ticks are still active.Delayed market data is available.SPX S&P 500 Stock Index/TOP/ALL, contract: Option(conId=681175922, symbol='SPX', lastTradeDateOrContractMonth='20240620', strike=5290.0, right='P', multiplier='100', exchange='SMART', currency='USD', localSymbol='SPX   240621P05290000', tradingClass='SPX')
Error 10090, reqId 62: Part of requested market data is not subscribed. Subscription-independent ticks are still active.Delayed market data is available.SPX S&P 500 Stock Index/TOP/ALL, contract: Option(conId=691599080, symbol='SPX', lastTradeDateOrContractMonth='20240718', strike=5255.0, right='P', multiplier='100', exchange='SMART', currency='USD', localSymbol='SPX   240719P05255000', tradingClass='SPX')
Error 10090, reqId 58: Part of requested market data is not subscribed. Subscription-independent ticks are still active.Delayed market data is available.SPX S&P 500 Stock Index/TOP

Ticker(contract=Option(conId=689492328, symbol='SPX', lastTradeDateOrContractMonth='20240620', strike=5255.0, right='P', multiplier='100', exchange='SMART', currency='USD', localSymbol='SPX   240621P05255000', tradingClass='SPX'), time=datetime.datetime(2024, 6, 3, 15, 34, 22, 660554, tzinfo=datetime.timezone.utc), marketDataType=3, modelGreeks=OptionComputation(tickAttrib=0, impliedVol=0.12209321934396355, delta=-0.42757615953857586, optPrice=45.88159447068788, pvDividend=5.616839500895427, gamma=0.0027314467801996863, vega=4.615974860260843, theta=-1.2020984631538112, undPrice=5270.5))

Error 10197, reqId 60: No market data during competing live session
Error 10197, reqId 78: No market data during competing live session
Error 10197, reqId 98: No market data during competing live session
Error 10197, reqId 96: No market data during competing live session
Error 10197, reqId 101: No market data during competing live session
Error 10197, reqId 97: No market data during competing live session
Error 10197, reqId 84: No market data during competing live session
Error 10197, reqId 69: No market data during competing live session
Error 10197, reqId 72: No market data during competing live session
Error 10197, reqId 82: No market data during competing live session
Error 10197, reqId 55: No market data during competing live session
Error 10197, reqId 85: No market data during competing live session
Error 10197, reqId 87: No market data during competing live session
Error 10197, reqId 58: No market data during competing live session
Error 10197, reqId 77: No market data during co

The option greeks are available from the ``modelGreeks`` attribute, and if there is a bid, ask resp. last price available also from ``bidGreeks``, ``askGreeks`` and ``lastGreeks``. For streaming ticks the greek values will be kept up to date to the current market situation.

In [11]:
ib.disconnect()